In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np
#import  seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn import model_selection, svm, metrics
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
breastCancer_data = pd.read_csv("./BreastCancer-Wisconsin/breast-cancer-wisconsin.data")
#print(breastCancer_data)

classes = ['malignant', 'benign']
features = ["clump_thickness", "cell_size_uniformity", "cell_shape_uniformity", "marginal_adhesion",
            "cell_size_epithelial", "bland_chromatin", "normal_nucleoli", "mitoses"]

X = breastCancer_data.drop(["id_number","bare_nuclei", "class"], axis=1)
Y = breastCancer_data["class"].replace([2, 4], [0, 1])
#print(X)
#print(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values, test_size = 0.2, random_state = 0, stratify = Y)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = X_train.reshape(559, 8,1)
X_test = X_test.reshape(140, 8, 1)
with open('ytest.npy', 'wb') as f:
    np.save(f, np.array(y_test))
f.close()
with open('xtest.npy', 'wb') as f:
    np.save(f, np.array(X_test))
f.close()    

In [ ]:
learning_rate = 0.00005
epochs = 50
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape = (8,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=learning_rate), loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
#history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
# Saving model
history = model.fit(X_train, y_train, epochs=epochs, callbacks=[checkpoint], validation_data=(X_test, y_test), verbose=1)

In [ ]:
def plot_learningCurve(history, epoch):
  # Plot training & validation accuracy values
  epoch_range = range(1, epoch+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

In [ ]:
plot_learningCurve(history, epochs)

In [ ]:
X_test2 = tf.cast(X_test, tf.float32)
predictions = model.predict(X_test2)
predictions2 = []
for p in predictions:
    if p <= 0.5:
        predictions2.append(0)
    else:
        predictions2.append(1)
accuracy = metrics.accuracy_score(y_test, predictions2)
print("Accuracy: ", accuracy*100, "%")
print("Features \t\t\t\t\tPredicted \t\t Target")
for (x, p, y) in zip(X_test, predictions2, y_test):
    x_features = []
    for x_attr in x:
        x_features.append(x_attr[0])
    print(x_features, "\t\t\t", classes[p], "\t\t", classes[y])

In [ ]:
model1 = load_model('model-023.model')
with open('xtest.npy', 'rb') as f:
    X_test = np.load(f)
f.close()
with open('ytest.npy', 'rb') as f:
    y_test = np.load(f)
f.close()
X_test2 = tf.cast(X_test, tf.float32)
predictions = model1.predict(X_test2)

predictions2 = []
for p in predictions:
    if p <= 0.5:
        predictions2.append(0)
    else:
        predictions2.append(1)
accuracy = metrics.accuracy_score(y_test, predictions2)
print("Accuracy: ", accuracy*100, "%")
print("Features \t\t\t\t\tPredicted \t\t Target")
for (x, p, y) in zip(X_test, predictions2, y_test):
    x_features = []
    for x_attr in x:
        x_features.append(x_attr[0])
    print(x_features, "\t\t\t", classes[p], "\t\t", classes[y])